In [ ]:
data = {
    'Distance' : ['Denis_Normal','Denis_FAR'], 
    'Lighting1' : ['Denis_Normal','Denis_Bright'],
    'Lighting2' : ['Denis_Normal','Denis_Dim_Lighting'], 
    'Age' : ['Brad_young','Brad_old'], 
    'Ethnicity1' : ['Jackie_chan_known', 'Jackie_chan_test1'], 
    'Ethnicity2' : ['Barack_Obama_Known','Barack_Obama_test'], 
    'Occlusion1' : ['Woman_Normal','Woman_Mask'],
    'Occlusion2' : ['Rana_Normal', 'Rana_Sunglass'], 
    'Occlusion3' : ['Rana_Normal', 'Rana_Multiple_Occlusion']
}

testCase = input('enter type of test please: ')

print('Starting comparison', data[testCase][0], 'and ', data[testCase][1])


from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import perf_counter
import sys
import numpy as np
import io
import json
import math
import matplotlib.image as mpimg
from matplotlib import rcParams
import matplotlib.pyplot as plt

file1 = 'known/'+data[testCase][0]+'.jpg'
file2 = 'test/'+data[testCase][1]+'.jpg'

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)

	print(len(results), 'faces have been found')

	#--------------------------- detecting the number of faces ------------------------------------
	if len(results) == 0:
		raise Exception('No faces detected')
	elif len(results) > 1:
		raise Exception('Multiple faces detected')		

	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
    
	return face_array

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	
	
	# create predication model - note that we have several 
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)

	return yhat

embedding1 = get_embeddings([file1])
vector1 = embedding1[0]

embedding2 = get_embeddings([file2])
vector2 = embedding2[0]
print('---------------------------Embeddings:----------------------')
print('The embedding from ' +data[testCase][0]+':', vector1)
print('The embedding from ' +data[testCase][1]+':', vector2)



def plot_initial_images(known_image, test_image):
    rcParams['figure.figsize'] = 13 ,10
    img_A = mpimg.imread(known_image)
    img_B = mpimg.imread(test_image)
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(img_A);
    ax[1].imshow(img_B);
    


def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:	
		print('>face is a Match (%.3f <= %.3f) = %.0f' % (score, thresh, math.floor((1-score)*100)), "%")
	else:
		print('>face is NOT a Match (%.3f <= %.3f) = %.0f' % (score, thresh, math.floor((1-score)*100)), "%")

plot_initial_images(file1, file2)

# comparing embeddings
print('---------------------------results:----------------------')
is_match(vector1, vector2)
print('The', data[testCase][0], 'and', data[testCase][1], 'were compared succesfully'  )
